This notebook will be converted into a distributable python script. The script will exist in a distributable file package with the following architecture:

```
Strain_modeler/   
|-- geometry/  
|-- dummies/
|-- output/  
|-- scripts.py
|-- README.md
```

1. The user will create a geometry folder, add a .xyz geometry of the optimized strained molecule, and as many .out gaussian output files from force calculations as necessary into the dummies folder.
2. A script will compile and average all of the .out files to create .tcl scripts in the output folder.
3. The user can then run the .tcl scripts in VMD to visualize each type of molecular strain.

In [1]:
from scripts import get_connectivity_data, get_atom_coords, load_geometry, create_key
from bond_scripts import map_forces, combine_dummies
from atom_scripts import vmd_vector_writer, atom_force_parse, normalize, map_atom_forces
import os

In [2]:
for file in os.listdir('geometry'):
    if file.endswith(".xyz"):
        geometry_filename = 'geometry/' + file

In [3]:
full_bond_forces = []
full_angle_forces = []
full_dihedral_forces = []
full_key = []
for file in os.listdir('dummies'):
    if file.endswith(".out"):
        a, b, c = map_forces(geometry_filename, file)
        for line in a:
            full_bond_forces.append(line)
        for line in b:
            full_angle_forces.append(line)
        for line in c:
            full_dihedral_forces.append(line)

In [4]:
combine_dummies(full_bond_forces, geometry_filename, "bond")
combine_dummies(full_angle_forces, geometry_filename, "angle")
combine_dummies(full_dihedral_forces, geometry_filename, "dihedral")

In [5]:
full_atom_forces = []
for file in os.listdir('dummies'):
    if file.endswith(".out"):
        a = map_atom_forces(geometry_filename, file)
        for line in a:
            full_atom_forces.append(line)

In [6]:
atom_coords = load_geometry(geometry_filename)

In [7]:
combined_atom_forces = []
for a in atom_coords:
    force = [0,0,0]
    for b in full_atom_forces:
        if a[2] == b[2][0] and a[3] == b[2][1] and a[4] == b[2][2]:
            force[0] += b[2][0]
            force[1] += b[2][1]
            force[2] += b[2][2]
    combined_atom_forces.append([force, a[2:]])

In [8]:
arrows = []
for index, line in enumerate(combined_atom_forces):
    arrows.append([((combined_atom_forces[index][0][0]**2)+(combined_atom_forces[index][0][1])**2+(combined_atom_forces[index][0][2])**2)])
    arrows[index].append(combined_atom_forces[index][1])
    arrows[index].append(combined_atom_forces[index][0])

In [9]:
norm_arrows = normalize(arrows)

In [10]:
vmd_vector_writer("test.tcl", norm_arrows, geometry_filename)